In [ ]:
import json
import time

from chromedriver_py import binary_path
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait

In [ ]:
def find(driver, selector):
    return driver.find_elements(By.CSS_SELECTOR, selector)

In [ ]:
def parse_profile(driver):
    profile = {
        "attributes": {},
        "chips": {},
        "questions": [],
        "images": [],
        "spotify": [],
    }
    
    # Html class tags for attributes.
    attribute_tags = {
        "about": ".encounters-story-about__text",
        "name": ".encounters-story-profile__name",
        "age": ".encounters-story-profile__age",
        "occupation": ".encounters-story-profile__occupation",
        "education": ".encounters-story-profile__education",
        "location": ".location-widget__town",
    }

    # Add attributes.
    for attribute, tag in attribute_tags.items():
        for e in find(driver, tag):
            profile["attributes"][attribute] = e.get_attribute("innerText")
            break

    # Add chips.
    for e in find(driver, ".pill__image"):
        chip = e.get_attribute("src")
        name = chip.split("profileChips_bff_")[1].split("v2")[0]
        profile["chips"][name] = e.get_attribute("alt")

    # Add questions.
    for e in find(driver, ".encounters-story-section--question"):
        profile["questions"].append(e.get_attribute("innerText"))
        
    # Add images.
    for e in find(driver, ".media-box__picture-image"):
        profile["images"].append(e.get_attribute("src"))

    # Add artists.
    for e in find(driver, ".spotify-widget__artist-name"):
        profile["spotify"].append(e.get_attribute("innerText"))

    return profile

In [ ]:
def should_dislike(driver):
    profile = parse_profile(driver)
    
    # Too few words in the about section.
    if len(profile["attributes"].get("about", "").split()) < 10:
        return True
    
    # Did not answer enough questions.
    if len(profile["questions"]) < 3:
        return True
    
    # Too few words in all questions combined.
    if sum([len(q.split("\n\n")[1].split()) for q in profile["questions"]]) < 10:
        return True
    
    # Not enough pictures.
    if len(profile["images"]) < 3:
        return True
    
    # No job.
    if "occupation" not in profile["attributes"]:
        return True
    
    return False

In [ ]:
# This automatically updates chromedriver.
driver = webdriver.Chrome(service=Service(binary_path))

In [ ]:
# Navigate to bumble.
driver.get("https://bumble.com/app")

In [ ]:
# The login process is a bit difficult to automate, so do it manually.

In [ ]:
while True:
    try:
        if should_dislike(driver):
            find(driver, ".encounters-action--dislike")[0].click()
        else:
            # Give time to manually like or dislike before checking again.
            time.sleep(1)
                 
    # This happens if the page is not loaded yet. Wait before trying again.
    except:
        time.sleep(1)